In [54]:
######## required downloads and useful instructions ##############
import warnings
warnings.filterwarnings('ignore')
!pip install biopython

In [55]:
######### imports #############
import os
import argparse
import sys
import time
import math

import torch
import torch.nn.functional as F
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from skimage.transform import resize
import tabulate
from torch.utils.data import DataLoader, ConcatDataset

from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, auc, roc_auc_score, average_precision_score
from itertools import product
import numpy as np 
import pandas as pd
import random

from scipy.stats import friedmanchisquare
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [56]:
'''
get all arguments, some are not changed by us and staded by the defult made by the aouthors of the paper.
The ones we changed mostly are:
  dataset - name of the database
  epochs - amount of epochs
  lr_init - the initial learning rate before momentum
  swa  - whether to use swa or just make regular sgd
  swa_start - from which ephocs to start the swa if swa is used
  swa_lr - the learning rate that will be once starting the SWA - in the regular it will be the same as the lr_init and in our improvement it wll be 0.02 bigger
  improved - whether to use swa improved 
'''
def get_args():
  parser = argparse.ArgumentParser(description='SGD/SWA/Improved training')
  parser.add_argument('--dir', type=str, default='.', help='training directory')
  parser.add_argument('--dataset', type=str, default='CIFAR10', help='dataset name')
  parser.add_argument('--data_path', type=str, default='.', metavar='PATH',
                      help='path to datasets location (default: .)')
  parser.add_argument('--batch_size', type=int, default=128, metavar='N', help='input batch size')
  parser.add_argument('--num_workers', type=int, default=4, metavar='N', help='number of workers')


  parser.add_argument('--epochs', type=int, default=200, metavar='N', help='number of epochs to train') 
  parser.add_argument('--lr_init', type=float, default=0.05, metavar='LR', help='initial learning rate')
  parser.add_argument('--momentum', type=float, default=0.9, metavar='M', help='SGD momentum')
  parser.add_argument('--wd', type=float, default=5e-4, help='weight decay')

  parser.add_argument('--swa', action='store_true', default=False, help='swa usage flag (default: off)')
  parser.add_argument('--swa_start', type=float, default=100, metavar='N', help='SWA start epoch number')
  parser.add_argument('--swa_lr', type=float, default=0.07, metavar='LR', help='SWA LR')
  parser.add_argument('--swa_c_epochs', type=int, default=1, metavar='N',
                      help='SWA model collection frequency/cycle length in epochs')
  parser.add_argument('--seed', type=int, default=1, metavar='S', help='random seed')

  parser.add_argument('--improved', action='store_true', default=False, help='improved usage flag')
  args, unknown = parser.parse_known_args()
  return args 

In [57]:
'''
VGG model definition, taken from the paper aouthors' repository 
ported from https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py
'''
__all__ = ['VGG16']


# reconstruct layers from model arhitecture 
def make_layers(cfg, batch_norm=False):
    layers = list()
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
          conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
          if batch_norm:
              layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
          else:
              layers += [conv2d, nn.ReLU(inplace=True)]
          in_channels = v
    return nn.Sequential(*layers)


cfg = {
    16: [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    19: [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M',
         512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, num_classes=10, depth=16, batch_norm=False):
        super(VGG, self).__init__()
        self.features = make_layers(cfg[depth], batch_norm)
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Linear(512, num_classes),
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                m.bias.data.zero_()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x


class Base:
    base = VGG
    args = list()
    kwargs = dict()

    transform_train = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

    #used for transforming gray scale image into RGB 32X32
    transform_train_gray_scale = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Resize(32),
        transforms.RandomCrop(32, padding=4),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1) ),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
       
    ])

    transform_test_gray_scale = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1) ),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

       
    ])  


class VGG16(Base):
    pass


In [58]:
def adjust_learning_rate(optimizer, lr):
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

#update weigth average of the SWA model according to the paper
def moving_average(net1, net2, alpha=1):
    for param1, param2 in zip(net1.parameters(), net2.parameters()):
        param1.data *= (1.0 - alpha)
        param1.data += param2.data * alpha

#constant learning rate schedule
def schedule(epoch, lr_init, num_epoch):
    t = (epoch) / (args.swa_start if args.swa else num_epoch)
    lr_ratio = args.swa_lr / lr_init if args.swa else 0.01
    if t <= 0.5:
        factor = 1.0
    elif t <= 0.9:
        factor = 1.0 - (1.0 - lr_ratio) * (t - 0.5) / 0.4
    else:
        factor = lr_ratio
    return lr_init * factor


# trains epoch using the VGG16 model and returns accuracy and loss metrics
def train_epoch(loader, model, criterion, optimizer):
    loss_sum = 0.0
    correct = 0.0

    model.train()

    for i, (input, target) in enumerate(loader):
        input = input.cuda()
        target = target.cuda()
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)

        output = model(input_var)
        loss = criterion(output, target_var)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_sum += loss.item() * input.size(0)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target_var.data.view_as(pred)).sum().item()

    return {
        'loss': loss_sum / len(loader.dataset),
        'accuracy': correct / len(loader.dataset) * 100.0,
    }

def get_metrics_one_vs_rest( y_true, predicted, proba):
  cm =  confusion_matrix(y_true, predicted)
  TN = cm[0][0]
  FP = cm[0][1]
  FN = cm[1][0]
  TP = cm[1][1] 
  # Overall accuracy for each class
  ACC = (TP+TN)/(TP+FP+FN+TN) # doesn't calculate as in the paper
  # true positive rate
  TPR = TP/(TP+FN)
  # false positive rate
  FPR = FP/(FP+TN)
  # Precision or positive predictive value
  Precision = TP/(TP+FP)
  #AUC
  AUC = roc_auc_score(y_true, proba)
  #PR-CURVE
  PR_CURVE = average_precision_score(y_true, proba)
  return { 'ACC': ACC, 'TPR': TPR, 'FPR':FPR, 'Precision': Precision, 'AUC':AUC, 'PR_CURVE':PR_CURVE}   

'''
calculates all metrics in a one vs rest approach.
returns dictionary of ACC, TPR, FPR, Precision, AUC, PR-CURVE values
'''
def one_vs_rest(y_true, predicted, proba, num_classes):
  metrics_all_classes = []
  for class_num in range(num_classes):
    y_true_class = [1 if y==class_num else 0 for y in y_true]
    predicted_class = [1 if pred==class_num else 0 for pred in predicted]
    one_class_values = list(get_metrics_one_vs_rest(y_true_class, predicted_class, proba[:, class_num]).values())
    metrics_all_classes.append(one_class_values)
  metrics_all_classes = np.mean(metrics_all_classes, axis=0)
  metrics =['ACC', 'TPR', 'FPR', 'Precision', 'AUC', 'PR_CURVE']
  metrics_results = dict()
  for i in range(len(metrics)):
    metrics_results[metrics[i]] = metrics_all_classes[i]
  return metrics_results


'''
evaluates the model and returns loss, accuracy and metrics dictionary which includes the following metrics:
Accuracy, TPR, FPR, Precision, AUC, Area under the Precision-Recall, Training time
and Inference time for 1000 instances
'''
def eval(loader, model, criterion, num_classes, training_time=0):
    loss_sum = 0.0
    correct = 0.0

    model.eval()
    y_true = torch.tensor([], dtype=torch.long, device=torch.device('cuda:0'))
    predicted = torch.tensor([], device=torch.device('cuda:0'))
    proba = torch.tensor([], device=torch.device('cuda:0'))

    inference_time = time.time()
    inference_times = []
    for i, (input, target) in enumerate(loader):
        input = input.cuda()
        target = target.cuda()
        input_var = torch.autograd.Variable(input)
        target_var = torch.autograd.Variable(target)
        output = model(input_var)
        loss = criterion(output, target_var)
        loss_sum += loss.item() * input.size(0)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target_var.data.view_as(pred)).sum().item()

        y_true = torch.cat((y_true, target_var), 0) #true lables
        predicted = torch.cat((predicted, output.data.max(1)[1]), 0) #predicted labels
        proba = torch.cat((proba, output.data), 0) #probabilites of certainty

        if (i + 1) % 8 == 0: #for calculating evaluation time for 1000 instances 
          inference_time = time.time() - inference_time 
          inference_times.append(inference_time)
          inference_time = time.time()

    predicted = predicted.cpu().numpy()
    y_true = y_true.cpu().numpy()
    proba = proba.cpu().numpy()
    
    #calculate all metrics using one vs rest approach 
    metrics = one_vs_rest(y_true, predicted, proba, num_classes)
    metrics['Training Time'] = training_time
    metrics['Inference Time'] =  np.mean(inference_times)

    accuracy =  correct / len(loader.dataset) * 100.0
    return {
        'loss': loss_sum / len(loader.dataset),
        'accuracy': accuracy,
        'metrics': metrics,
    }


In [59]:
'''
The function is activated for hyper-parameters optimization only.
Gets hyper-parameters configuration and returns accuracy value.
'''
def find_accuracy_for_specific_hyperparameters(config , model_cfg, loaders):      
  print('Find best parameters')
  model = model_cfg.base(*model_cfg.args, num_classes=num_classes, **model_cfg.kwargs)
  model.cuda()

  num_epoch = config[1]
  lr_init = config[0]

  criterion = F.cross_entropy
  optimizer = torch.optim.SGD(
      model.parameters(),
      lr=lr_init,
      momentum=args.momentum,
      weight_decay=args.wd
  )
  start_epoch = 0

  for epoch in range(start_epoch, num_epoch):
      lr = schedule(epoch, lr_init, num_epoch)
      adjust_learning_rate(optimizer, lr)
      train_res = train_epoch(loaders['train'], model, criterion, optimizer)
      test_res = eval(loaders['val'], model, criterion, num_classes)

      #printing per epoch 
      values = [epoch + 1, lr, train_res['loss'], train_res['accuracy'], test_res['loss'], test_res['accuracy']]
      table = tabulate.tabulate([values], ['ep', 'lr', 'tr_loss', 'tr_acc', 'te_loss', 'te_acc'], tablefmt='simple', floatfmt='8.4f')
      print(table)

  return test_res['accuracy'] 

In [67]:
'''
The function evaluates the model. 
Applies conventional SGD or SWA algorithms according to argument args.swa
'''
def evaluate_model(lr_init, num_epoch, train_loader, test_loader, num_classes):      
  print('Evaluate model')
  model = model_cfg.base(*model_cfg.args, num_classes=num_classes, **model_cfg.kwargs)
  model.cuda()

  if args.swa:
      swa_model = model_cfg.base(*model_cfg.args, num_classes=num_classes, **model_cfg.kwargs)
      swa_model.cuda()
      swa_n = 0

  criterion = F.cross_entropy
  optimizer = torch.optim.SGD(
      model.parameters(),
      lr=lr_init,
      momentum=args.momentum,
      weight_decay=args.wd
  )
  start_epoch = 0
  
  for epoch in range(start_epoch, num_epoch):
      lr = schedule(epoch, lr_init, num_epoch)
      adjust_learning_rate(optimizer, lr)
      training_time = time.time()
      train_res = train_epoch(train_loader, model, criterion, optimizer)
      training_time = time.time() - training_time
      test_res = eval(test_loader, model, criterion, num_classes, training_time)

      if not args.swa:
         print(f'{test_res},')

      #if we are using swa and we are at the swa phase and (we are using constant learning rate schedule or we are at a start of a cycle in a cyclic schedule)
      if args.swa and (epoch + 1) >= args.swa_start and (epoch + 1 - args.swa_start) % args.swa_c_epochs == 0:
          # calculate the avarage of the weights
          moving_average(swa_model, model, 1.0 / (swa_n + 1))
          swa_n += 1
          # evaluate test-set performance 
          swa_res = eval(test_loader, swa_model, criterion, num_classes, training_time)
          print(f'{swa_res},')  
          
      elif args.swa:
        print(f'epoch = {epoch}/{num_epoch},')       
      
  if args.swa:
    return swa_res
  else:
    return test_res

In [61]:
def hyper_parameters_optimization(dataset):
  if args.swa:  # optimization is applied only on sgd 
      return [args.lr_init, args.epochs]
  best_parameters = None
  best_accuracy = 0
  # outer k-fold Cross Validation 
  kfold = KFold(n_splits=10, shuffle=True)
  for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
      train_subs = torch.utils.data.dataset.Subset(dataset, train_ids)
      #hyper-parameters space
      config = {
          'lr_init': [x / 100.0 for x in range(1, 11)],
          'num_epochs': list(range(50, 300, 25))
      }
      all_options = list(product(config['lr_init'], config['num_epochs']))
      # select randomly 50 times
      for _ in range(50):
          curr_config = random.choice(all_options)
          print(f'choice {curr_config}')
          # inner k-fold for hyperparameters optimization
          mean_acc = 0
          inner_kfold = KFold(n_splits=3, shuffle=True)
          for fold, (in_train_ids, in_val_ids) in enumerate(inner_kfold.split(train_subs)):
              inner_train = torch.utils.data.dataset.Subset(dataset, in_train_ids)
              inner_val = torch.utils.data.dataset.Subset(dataset, in_val_ids)

              # Define data loaders for training and validation data in this fold
              inner_train_loader = torch.utils.data.DataLoader(
                  inner_train,
                  batch_size=args.batch_size,
                  num_workers=args.num_workers,
                  shuffle=True)

              inner_val_loader = torch.utils.data.DataLoader(
                  inner_val,
                  batch_size=args.batch_size,
                  shuffle=False,
                  num_workers=args.num_workers)

              loaders = {'train': inner_train_loader, 'val': inner_val_loader}
              # train the network with the chosen hyperparameters
              acc = find_accuracy_for_specific_hyperparameters(curr_config, model_cfg, loaders)
              mean_acc += acc

          # determine the mean accurecy to be mean_acc / 3
          mean_acc = mean_acc / 3
          if best_accuracy < mean_acc:
              best_accuracy = mean_acc
              # choose the best to be the one with highest mean accrucy over the 3 inner folds.
              best_parameters = curr_config
          print(f'best parameters {best_parameters}')

  return best_parameters 

In [62]:
'''
Downloads the desired dataset according to args.dataset, applies transformation if needed, determined number of classes
'''
def download_dataset():
  if args.dataset == 'EMNIST':
    train_set = ds(path, train=True, download=True, transform=model_cfg.transform_train_gray_scale, split='digits')
    test_set = ds(path, train=False, download=True, transform=model_cfg.transform_test_gray_scale, split='digits')
  elif args.dataset == 'MNIST' or args.dataset == 'USPS' or args.dataset == 'FashionMNIST' or args.dataset == 'KMNIST' or args.dataset == 'QMNIST':
    train_set = ds(path, train=True, download=True, transform=model_cfg.transform_train_gray_scale)
    test_set = ds(path, train=False, download=True, transform=model_cfg.transform_test_gray_scale)
  elif args.dataset == 'STL10' or args.dataset == 'SVHN':
    train_set = ds(path, download= True ,transform=model_cfg.transform_train)
  elif args.dataset == 'SEMEION':
     train_set = ds(path, download=True, transform=model_cfg.transform_train_gray_scale)
  else:# CIFAR10, CIFAR100
    train_set = ds(path, train=True, download=True, transform=model_cfg.transform_train)
    test_set = ds(path, train=False, download=True, transform=model_cfg.transform_test)

  if args.dataset == 'CIFAR100':
    num_classes = 100 
  else:
    num_classes = 10 
  if args.dataset == 'STL10' or args.dataset == 'SVHN' or args.dataset == 'SEMEION':
    dataset = train_set
  else:
    dataset = ConcatDataset([train_set, test_set])
  return dataset, num_classes

In [63]:
def check_if_nan(value):
  for val_ind in range(len(value))[4:]:
      if math.isnan(value[val_ind]):
        value[val_ind] = 0 
  return value

In [64]:
'''
Gets three models and return whether they have the same distribution
'''
def friedman_test(sgd, swa, improved):
  # compare samples
  return friedmanchisquare(sgd, swa, improved)

'''
Get three models and calculates the differences between them in terms of p-value and mean difference
'''
def post_hoc_test(sgd, swa, improved):
  scores = sgd
  scores.extend(swa)
  scores.extend(improved)
  #create DataFrame to hold data
  df = pd.DataFrame({'score': scores,
                    'group': np.repeat(['SGD', 'SWA', 'Improved'], repeats=10)}) 
  # perform Tukey's test
  tukey = pairwise_tukeyhsd(endog=df['score'], groups=df['group'], alpha=0.05)
  #display results
  print(tukey)

'''
Get the file name of the file with the results of the 3 algorithms,
the wanted dataset and the wanted metric on which we want to test if the preformances were the same
and prints the results.
'''
def significant_test(file_name, dataset_name, metric):
  # open the xlsx file with the results of the datasets
  df = pd.read_excel(file_name, header=None).iloc[:,:-1]
  df.columns= ['Dataset Name', 'Algorithm Name', 'Cross Validation [1-10]', 'HyperParamaters Values', 'ACC', 'TPR', 'FPR', 'Precision','AUC', 'PR-CURVE', 'Training Time', 'Inference Time']
  # take the results of the metric in the dataset for each algorithm
  sgd = df.loc[(df['Dataset Name'] == dataset_name) & (df['Algorithm Name'] == 'SGD')][metric].values.tolist()
  swa = df.loc[(df['Dataset Name'] == dataset_name) & (df['Algorithm Name'] == 'SWA')][metric].values.tolist()
  improved =  df.loc[(df['Dataset Name'] == dataset_name) & (df['Algorithm Name'] == 'improved')][metric].values.tolist()
  # apply the firdeman test
  stat, p = friedman_test(sgd, swa, improved)
  
  # interpret - if the distributions are different then apply also post_hoc test to measure the difference
  alpha = 0.05
  if p > alpha:
    print('Same distributions (fail to reject H0)')
  else:
    print('Different distributions (reject H0)')
    post_hoc_test(sgd, swa, improved)


In [ ]:
"""
  MAIN
"""
#parse args
args = get_args()

# set up directory
os.makedirs(args.dir, exist_ok=True)
with open(os.path.join(args.dir, 'command.sh'), 'w') as f:
    f.write(' '.join(sys.argv))
    f.write('\n')

# set up cuda with seed
torch.backends.cudnn.benchmark = True
torch.manual_seed(args.seed)
torch.cuda.manual_seed(args.seed)

# set up model 
model_cfg = VGG16()  

#chose the algorithm to run
if args.improved:
  print('Using SWA Improved')
  alg_name = 'improved'
elif args.swa:
  print('Using SWA')
  alg_name = 'SWA'
else:
  print('Using SGD')
  alg_name = 'SGD'

#loading dataset from torchvision package 
print('Loading dataset %s from %s' % (args.dataset, args.data_path))
ds = getattr(torchvision.datasets, args.dataset)  
path = os.path.join(args.data_path, args.dataset.lower())

dataset, num_classes = download_dataset()


values = []
columns = ['Dataset Name', 'Algorithm Name', 'Cross Validation [1-10]', 'HyperParamaters Values', 'ACC', 'TPR', 'FPR', 'Precision','AUC', 'PR-CURVE', 'Training Time', 'Inference Time']

#choose the best parameters by the hyper parameters optimization
best_parameters = hyper_parameters_optimization(dataset)

# outer k-fold Cross Validation 
kfold = KFold(n_splits=10, shuffle=True)
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
    train_subs = torch.utils.data.dataset.Subset(dataset, train_ids)
    test_subs = torch.utils.data.dataset.Subset(dataset, test_ids)

    train_loader = torch.utils.data.DataLoader(
    train_subs,
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    shuffle=True)
        
    test_loader = torch.utils.data.DataLoader(
    test_subs,
    batch_size=args.batch_size,
    num_workers=args.num_workers,
    shuffle=True)

    # train the model with the best parameters chosen and evaluate this results on the K outer datasets
    res = evaluate_model(best_parameters[0], best_parameters[1], train_loader, test_loader, num_classes)
    value = [args.dataset, alg_name, fold+1, best_parameters] + list(res['metrics'].values())
    value = check_if_nan(value)
    values.append(value)

# put the reults into csv file. later,  for the submission and statistics tests we transformed it to an arranged xlsx file !!!
df_results = pd.DataFrame(data=values, columns = columns)
if os.path.isfile('results.csv'):
  df_results.to_csv('results.csv', mode='a', index=False)
else:
  df_results.to_csv('results.csv', index=False)
print(df_results.to_string())

In [70]:
# run the significant test for the datasets we have the results for
for name in ['CIFAR10', 'CIFAR100', 'EMNIST', 'FashionMNIST', 'KMNIST','MNIST', 'QMNIST', 'STL10', 'USPS', 'SVHN', 'SEMEION']:
  print(f'Dataset {name}')
  significant_test('results.xlsx',name, 'ACC')

Dataset CIFAR10
Different distributions (reject H0)
 Multiple Comparison of Means - Tukey HSD, FWER=0.05 
 group1  group2 meandiff p-adj  lower   upper  reject
-----------------------------------------------------
Improved    SGD   0.0061  0.001 0.0056  0.0066   True
Improved    SWA   0.0055  0.001  0.005   0.006   True
     SGD    SWA  -0.0005 0.0266 -0.001 -0.0001   True
-----------------------------------------------------
Dataset CIFAR100
Different distributions (reject H0)
Multiple Comparison of Means - Tukey HSD, FWER=0.05
 group1  group2 meandiff p-adj lower  upper  reject
---------------------------------------------------
Improved    SGD   0.0071 0.001 0.0067 0.0074   True
Improved    SWA   0.0092 0.001 0.0088 0.0095   True
     SGD    SWA   0.0021 0.001 0.0018 0.0024   True
---------------------------------------------------
Dataset EMNIST
Different distributions (reject H0)
 Multiple Comparison of Means - Tukey HSD, FWER=0.05  
 group1  group2 meandiff p-adj   lower   upper 